In [ ]:
# D-Wave Leap Authentication
import os
from dwave.cloud import Client

def setup_dwave_config():
    """
    Setup D-Wave configuration for Leap access.
    Make sure you have your API token set up.
    """
    # Option 1: Set environment variable
    # os.environ['DWAVE_API_TOKEN'] = 'your_api_token_here'

    # Option 2: Use dwave config file (recommended)
    # Run: dwave config create in terminal to set up interactively

    # Test connection
    try:
        with Client.from_config() as client:
            print("✓ Successfully connected to D-Wave Leap")
            return True
    except Exception as e:
        print(f"✗ Connection failed: {e}")
        print("Please set up your D-Wave API token using 'dwave config create'")
        return False

In [2]:
# Test D-Wave connection
setup_dwave_config()

✗ Connection failed: API token not defined
Please set up your D-Wave API token using 'dwave config create'


False

In [1]:
# ==============================================================================
# SETUP AND INSTALLATIONS
# ==============================================================================
print("--- Installing D-Wave libraries... ---")
%pip install dwave-ocean-sdk --upgrade -q

# from google.colab import drive
# import sys, math

# print("\n--- Mounting Google Drive and setting up project path... ---")
# drive.mount('/content/drive', force_remount=True)

# # IMPORTANT: Make sure your project files are in this folder on your Google Drive
# project_path = '/content/drive/MyDrive/qintern-vrp/New-VRP'

# # Add the project path to the system's Python path so it can find your modules
# if project_path not in sys.path:
#     sys.path.append(project_path)
# print(f"Project path '{project_path}' added to system path.")

--- Installing D-Wave libraries... ---
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.2/161.2 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 76.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 84.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.3/239.3 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.9 MB/s eta 0:00:00


In [ ]:
# ==============================================================================
# MAIN EXPERIMENT SCRIPT
# Runs a predefined list of experiments to systematically verify solver performance.
# ==============================================================================
import numpy as np
import time
import math
import pandas as pd

from input import read_solomon
from vrp_problem import VRPProblem
from vrp_solvers import FullQuboSolver, AveragePartitionSolver

print("\n\n--- Starting VRP Solver Experiment Suite ---")

# ==============================================================================
# EXPERIMENT CONFIGURATION
# ==============================================================================
SOLOMON_FILE_PATH = 'c101.txt' # Adjust this path as needed
LIMIT_RADIUS = 1

# ==============================================================================
# This list is designed to replicate every FQS and APS test case from
# Table 3 of the Borowski, Gora, et al. paper.
# NOTE: The customer data is drawn from 'c101.txt', which differs from the
# paper's custom dataset. Expect different cost results, but look for
# similar performance *trends* between FQS and APS.
# ==============================================================================
experiments = [
    # --- "small" test cases from paper's Table 3 ---
    {'name': 'small-0 (2 cust, 1 veh)', 'customers': 2, 'vehicles': 1},
    {'name': 'small-0 (2 cust, 2 veh)', 'customers': 2, 'vehicles': 2},

    {'name': 'small-1 (2 cust, 1 veh)', 'customers': 2, 'vehicles': 1},
    {'name': 'small-1 (2 cust, 2 veh)', 'customers': 2, 'vehicles': 2},
    {'name': 'small-1 (2 cust, 3 veh)', 'customers': 2, 'vehicles': 3},

    {'name': 'small-2 (2 cust, 1 veh)', 'customers': 2, 'vehicles': 1},
    {'name': 'small-2 (2 cust, 2 veh)', 'customers': 2, 'vehicles': 2},
    {'name': 'small-2 (2 cust, 3 veh)', 'customers': 2, 'vehicles': 3},

    {'name': 'small-3 (1 cust, 1 veh)', 'customers': 1, 'vehicles': 1},
    {'name': 'small-3 (1 cust, 2 veh)', 'customers': 1, 'vehicles': 2},
    {'name': 'small-3 (1 cust, 3 veh)', 'customers': 1, 'vehicles': 3},

    {'name': 'small-4 (2 cust, 1 veh)', 'customers': 2, 'vehicles': 1},
    {'name': 'small-4 (2 cust, 2 veh)', 'customers': 2, 'vehicles': 2},
    {'name': 'small-4 (2 cust, 3 veh)', 'customers': 2, 'vehicles': 3},

    {'name': 'small-5 (5 cust, 1 veh)', 'customers': 5, 'vehicles': 1},
    {'name': 'small-5 (5 cust, 2 veh)', 'customers': 5, 'vehicles': 2},
    {'name': 'small-5 (5 cust, 3 veh)', 'customers': 5, 'vehicles': 3},

    {'name': 'small-6 (6 cust, 1 veh)', 'customers': 6, 'vehicles': 1},
    {'name': 'small-6 (6 cust, 2 veh)', 'customers': 6, 'vehicles': 2},
    {'name': 'small-6 (6 cust, 3 veh)', 'customers': 6, 'vehicles': 3},

    {'name': 'small-7 (5 cust, 1 veh)', 'customers': 5, 'vehicles': 1},
    {'name': 'small-7 (5 cust, 2 veh)', 'customers': 5, 'vehicles': 2},
    {'name': 'small-7 (5 cust, 3 veh)', 'customers': 5, 'vehicles': 3},

    {'name': 'small-8 (4 cust, 1 veh)', 'customers': 4, 'vehicles': 1},
    {'name': 'small-8 (4 cust, 2 veh)', 'customers': 4, 'vehicles': 2},
    {'name': 'small-8 (4 cust, 3 veh)', 'customers': 4, 'vehicles': 3},

    {'name': 'small-9 (6 cust, 1 veh)', 'customers': 6, 'vehicles': 1},
    {'name': 'small-9 (6 cust, 2 veh)', 'customers': 6, 'vehicles': 2},
    {'name': 'small-9 (6 cust, 3 veh)', 'customers': 6, 'vehicles': 3},

    # --- "medium" test cases from paper's Table 3 ---
    {'name': 'medium-0 (20 cust, 1 veh)', 'customers': 20, 'vehicles': 1},
    {'name': 'medium-0 (20 cust, 2 veh)', 'customers': 20, 'vehicles': 2},
    {'name': 'medium-0 (20 cust, 3 veh)', 'customers': 20, 'vehicles': 3},

    {'name': 'medium-1 (26 cust, 1 veh)', 'customers': 26, 'vehicles': 1},
    {'name': 'medium-1 (26 cust, 2 veh)', 'customers': 26, 'vehicles': 2},
    {'name': 'medium-1 (26 cust, 3 veh)', 'customers': 26, 'vehicles': 3},

    {'name': 'medium-2 (27 cust, 1 veh)', 'customers': 27, 'vehicles': 1},
    {'name': 'medium-2 (27 cust, 2 veh)', 'customers': 27, 'vehicles': 2},
    {'name': 'medium-2 (27 cust, 3 veh)', 'customers': 27, 'vehicles': 3},

    {'name': 'medium-3 (24 cust, 1 veh)', 'customers': 24, 'vehicles': 1},
    {'name': 'medium-3 (24 cust, 2 veh)', 'customers': 24, 'vehicles': 2},
    {'name': 'medium-3 (24 cust, 3 veh)', 'customers': 24, 'vehicles': 3},

    {'name': 'medium-4 (25 cust, 1 veh)', 'customers': 25, 'vehicles': 1},
    {'name': 'medium-4 (25 cust, 2 veh)', 'customers': 25, 'vehicles': 2},
    {'name': 'medium-4 (25 cust, 3 veh)', 'customers': 25, 'vehicles': 3},

    {'name': 'medium-5 (25 cust, 1 veh)', 'customers': 25, 'vehicles': 1},
    {'name': 'medium-5 (25 cust, 2 veh)', 'customers': 25, 'vehicles': 2},
    {'name': 'medium-5 (25 cust, 3 veh)', 'customers': 25, 'vehicles': 3},

    {'name': 'medium-6 (20 cust, 1 veh)', 'customers': 20, 'vehicles': 1},
    {'name': 'medium-6 (20 cust, 2 veh)', 'customers': 20, 'vehicles': 2},
    {'name': 'medium-6 (20 cust, 3 veh)', 'customers': 20, 'vehicles': 3},

    {'name': 'medium-7 (14 cust, 1 veh)', 'customers': 14, 'vehicles': 1},
    {'name': 'medium-7 (14 cust, 2 veh)', 'customers': 14, 'vehicles': 2},
    {'name': 'medium-7 (14 cust, 3 veh)', 'customers': 14, 'vehicles': 3},

    {'name': 'medium-8 (17 cust, 1 veh)', 'customers': 17, 'vehicles': 1},
    {'name': 'medium-8 (17 cust, 2 veh)', 'customers': 17, 'vehicles': 2},
    {'name': 'medium-8 (17 cust, 3 veh)', 'customers': 17, 'vehicles': 3},

    {'name': 'medium-9 (15 cust, 1 veh)', 'customers': 15, 'vehicles': 1},
    {'name': 'medium-9 (15 cust, 2 veh)', 'customers': 15, 'vehicles': 2},
    {'name': 'medium-9 (15 cust, 3 veh)', 'customers': 15, 'vehicles': 3},
]

# This list will store the summary of each experiment for the final table.
all_results = []

# ==============================================================================
# MAIN EXPERIMENT LOOP
# ==============================================================================
for test_config in experiments:

    NUM_CUSTOMERS = test_config['customers']
    NUM_VEHICLES = test_config['vehicles']

    print("\n\n" + "#"*70)
    print(f"###   STARTING TEST: {test_config['name']} ({NUM_CUSTOMERS} Customers, {NUM_VEHICLES} Vehicles)   ###")
    print("#"*70)

    # --- DATA LOADING AND PROBLEM SETUP ---
    print("\n--- Preparing the Vehicle Routing Problem ---")

    problem_data = read_solomon(SOLOMON_FILE_PATH, NUM_CUSTOMERS)
    vehicle_capacity = problem_data['capacities'][0]
    problem_data['capacities'] = np.array([vehicle_capacity] * NUM_VEHICLES)

    problem = VRPProblem(
        sources=problem_data['sources'],
        costs=problem_data['costs'],
        time_costs=problem_data['time_costs'],
        capacities=problem_data['capacities'],
        dests=problem_data['dests'],
        weights=problem_data['weights']
    )
    print(f"Problem created successfully for {len(problem.dests)} customers and {len(problem.capacities)} vehicles.")

    # --- QUBO FORMULATION PARAMETERS ---
    order_const = 1.0
    A1 = 1.0
    A2 = 10**7 # Using the exact (large) penalty from the paper
    print(f"\\nQUBO Parameters: A1 (objective) = {A1}, A2 (constraints) = {A2}")


    # --- EXPERIMENT 1 - FULL QUBO SOLVER (FQS) ---
    print("\n" + "="*60)
    print("--- RUNNING EXPERIMENT: FullQuboSolver (FQS) ---")
    print("="*60)

    fqs_solver = FullQuboSolver(problem)
    print("Solving with classical Simulated Annealing...")
    start_time_fqs = time.time()
    fqs_solution = fqs_solver.solve(A1, A2, solver_type='simulated')
    end_time_fqs = time.time()
    fqs_run_time = end_time_fqs - start_time_fqs

    print("\n--- FQS RESULTS ---")
    print(f"Solver run time: {fqs_run_time:.2f} seconds")
    fqs_solution.description()
    fqs_cost = fqs_solution.total_cost()
    fqs_valid = fqs_solution.check()

    # --- EXPERIMENT 2 - AVERAGE PARTITION SOLVER (APS) ---
    print("\n" + "="*60)
    print("--- RUNNING EXPERIMENT: AveragePartitionSolver (APS) ---")
    print("="*60)

    aps_solver = AveragePartitionSolver(problem)
    print("Solving with classical Simulated Annealing...")
    start_time_aps = time.time()
    aps_solution = aps_solver.solve(A1, A2, solver_type='simulated', limit_radius=LIMIT_RADIUS)
    end_time_aps = time.time()
    aps_run_time = end_time_aps - start_time_aps

    print("\n--- APS RESULTS ---")
    print(f"Solver run time: {aps_run_time:.2f} seconds")
    aps_solution.description()
    aps_cost = aps_solution.total_cost()
    aps_valid = aps_solution.check()

    # --- STORE RESULTS FOR THIS TEST CONFIGURATION ---
    result_summary = {
        'Test Name': test_config['name'],
        'Customers': NUM_CUSTOMERS,
        'Vehicles': NUM_VEHICLES,
        'FQS Valid': '✅' if fqs_valid else '❌',
        'FQS Cost': f"{fqs_cost:.2f}",
        'FQS Time (s)': f"{fqs_run_time:.2f}",
        'APS Valid': '✅' if aps_valid else '❌',
        'APS Cost': f"{aps_cost:.2f}",
        'APS Time (s)': f"{aps_run_time:.2f}"
    }
    all_results.append(result_summary)


# ==============================================================================
# FINAL SUMMARY OF ALL EXPERIMENTS
# ==============================================================================
print("\n\n" + "#"*70)
print("###                  FINAL SUMMARY OF ALL EXPERIMENTS                  ###")
print("#"*70)

# Use pandas to create and display a clean table
df_results = pd.DataFrame(all_results)
print(df_results.to_string())



--- Starting VRP Solver Experiment Suite ---


######################################################################
###   STARTING TEST: small-0 (2 cust, 1 veh) (2 Customers, 1 Vehicles)   ###
######################################################################

--- Preparing the Vehicle Routing Problem ---
Problem created successfully for 2 customers and 1 vehicles.
\nQUBO Parameters: A1 (objective) = 1.0, A2 (constraints) = 10000000

--- RUNNING EXPERIMENT: FullQuboSolver (FQS) ---
Solving with classical Simulated Annealing...

--- FQS RESULTS ---
Solver run time: 0.00 seconds
Solution Routes:
  Vehicle 0: 0 -> 1 -> 2 -> 0

Total Cost: 41.30
Is Solution Valid: True

--- RUNNING EXPERIMENT: AveragePartitionSolver (APS) ---
Solving with classical Simulated Annealing...

--- APS RESULTS ---
Solver run time: 0.00 seconds
Solution Routes:
  Vehicle 0: 0 -> 2 -> 1 -> 0

Total Cost: 41.30
Is Solution Valid: True


######################################################################


In [6]:
# Testing all solver types

solver_types = ['simulated', 'exact', 'hybrid']  # Add 'qpu' if you have access

for solver_type in solver_types:
    print(f"\n=== Testing {solver_type} solver ===")
    try:
        # Use a small problem for testing
        test_problem = read_solomon(SOLOMON_FILE_PATH, 3)  # Only 3 customers right now
        # ... run your solver test here
        print(f"✓ {solver_type} solver working correctly")
    except Exception as e:
        print(f"✗ {solver_type} solver failed: {e}")


=== Testing simulated solver ===
✓ simulated solver working correctly

=== Testing exact solver ===
✓ exact solver working correctly

=== Testing hybrid solver ===
✓ hybrid solver working correctly
